# Tiny A11y Model Fine-tune & Test Notebook
This notebook fine-tunes DeepSeek-Coder on WCAG + MDN accessibility datasets and allows testing your LoRA model.

In [ ]:
# Install required packages
!pip install --upgrade pip
!pip install datasets transformers peft huggingface_hub accelerate

In [ ]:
# Login to Hugging Face Hub
from huggingface_hub import notebook_login
notebook_login()  # Paste your token when prompted

In [ ]:
# Load dataset from Hugging Face
from datasets import load_dataset
dataset = load_dataset('younglim/a11y-dataset', split='train')
print(dataset[0])

In [ ]:
# Load base model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, PeftModel

base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)

# LoRA config (for training, optional)
peft_config = LoraConfig(r=8, lora_alpha=16, target_modules=['q_proj','v_proj'])
model = get_peft_model(base_model, peft_config)

In [ ]:
# Tokenize dataset
def tokenize(example):
    return tokenizer(example['text'], truncation=True, padding='max_length', max_length=512)

dataset = dataset.map(tokenize, batched=True)

In [ ]:
# Setup Trainer and TrainingArguments
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=1e-4,
    fp16=True,
    push_to_hub=True,
    hub_model_id='younglim/tiny-a11y-model'
)

trainer = Trainer(model=model, args=training_args, train_dataset=dataset)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Push the fine-tuned LoRA model to Hugging Face Hub
trainer.push_to_hub()

In [ ]:
# === TEST THE LoRA MODEL ===
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load base model and attach LoRA adapter
base_model_name = 'deepseek-ai/deepseek-coder-1.3b-instruct'
lora_model_name = 'younglim/tiny-a11y-model'

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = PeftModel.from_pretrained(base_model, lora_model_name)

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Test prompt
prompt = "Explain why alt text is important for images."
inputs = tokenizer(prompt, return_tensors='pt')
outputs = model.generate(**inputs, max_new_tokens=150, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))